# Audio Model training
En este notebook se realizará el entrenamiento de los modelos de aprendizaje automático relacionados con el audio.
## Imports

In [4]:
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
import random as rnd
from joblib import dump

from audio_model_training import *

## Parámetros
Indicamos los parámetros del modelo que se quiere entrenar.

In [5]:
project_dir = "/home/daniel/PycharmProjects/tfg-tinyml"
saved_models_dir = f"{project_dir}/saved models/audio"
label_index_dir = f"{saved_models_dir}/label index mappings"

data_origin = MICRO

if data_origin == MICRO:
    data_dir = f"{project_dir}/samples/microcontroller/audio/train"
    origin_name = "Micro"
    epochs = 2000

else:
    data_dir = f"{project_dir}/samples/external/audio/train"
    origin_name = "Ext"
    epochs = 50

# input_shape =

model_name = f"{origin_name}{DATA_TYPE}DataModel"
log_dir = f"{saved_models_dir}/summaries/{model_name}"
model_dir = f"{saved_models_dir}/tensorflow/{model_name}"

validation_percentage = 20.

initial_learning_rate = 0.001
decay = (initial_learning_rate / epochs) * 1.5

class_names = ["no", "unknown", "silence", "yes"]


tf.random.set_seed(SEED)
np.random.seed(SEED)
rnd.seed(SEED)

## Carga de datos
Cargamos las muestras de audio en forma de dataset. Aprovechamos para guardar la correspondencia entre labels y nombres
de clase.

In [6]:
class_indexes = {}
class_list_text = "Classes:"
for index, name in enumerate(class_names):
    class_list_text += f"\n\t{index}: {name}"
    class_indexes[name] = index
print(class_list_text)
with open(f"{label_index_dir}/{model_name}.txt", "wt") as text_file:
    text_file.write(class_list_text)

dump(class_names, f"{label_index_dir}/{model_name}.joblib")

train_files, validation_files = get_audio_files(data_dir, validation_percentage, class_names)

Classes:
	0: no
	1: unknown
	2: silence
	3: yes

Particionando datos:
	- Directorio: /home/daniel/PycharmProjects/tfg-tinyml/samples/microcontroller/audio/train
	- Porcetage validación: 20.0%
	- 94 muestras de la clase no donde 75 son para entrenamiento y 19 son para validación.
	- 81 muestras de la clase unknown donde 64 son para entrenamiento y 17 son para validación.
	- 84 muestras de la clase silence donde 67 son para entrenamiento y 17 son para validación.
	- 102 muestras de la clase yes donde 82 son para entrenamiento y 20 son para validación.
[(1, '/home/daniel/PycharmProjects/tfg-tinyml/samples/microcontroller/audio/train/unknown/unknown59.wav'), (3, '/home/daniel/PycharmProjects/tfg-tinyml/samples/microcontroller/audio/train/yes/yes94.wav'), (3, '/home/daniel/PycharmProjects/tfg-tinyml/samples/microcontroller/audio/train/yes/yes16.wav'), (0, '/home/daniel/PycharmProjects/tfg-tinyml/samples/microcontroller/audio/train/no/no92.wav'), (0, '/home/daniel/PycharmProjects/tfg-tinyml/